In [1]:
!pip install tensorflow_text
import tensorflow_text as text  # Registers the ops.

# After running this cell, we have to restart the Kernel and after restarting run this cell once again!

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.0.5
    Uninstalling keras-3.0.5:
      Successfully uninstalled keras-3.0.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


2024-04-07 03:41:14.153217: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 03:41:14.153351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 03:41:14.264693: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
!pip install contractions
!pip install transformers
!pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.0 MB/s eta 0:00:00


In [4]:
import spacy
import numpy as np
import pandas as pd
import contractions
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

data = pd.read_csv('/kaggle/input/memes-data/data.csv')
df = data.sample(frac=0.8, random_state=42)
test = data.drop(df.index)
df.head(7)

,text,img,label,Caption
6252,i did nazi that bird fly by did you?,img/63915.png,0,a black and white photo of a man with a mustache
4684,best holiday gift for muslims,img/37864.png,0,there is a man and woman sitting at a table wi...
1731,found a picture of your mother's womb,img/90625.png,1,there is a baby that is laying down on a blanket
4742,bruce jenner's cat,img/12807.png,0,arafed cat sitting in front of a building with...
4521,i do not want to hear anymore about gun contro...,img/26049.png,0,arafed image of a man in a suit and tie with a...
6340,"normie: ""your meme's are racist and offensive,...",img/24537.png,1,there is a kid that is playing with a spongebob
576,it's time to kill the british and dump tea int...,img/35216.png,1,someone told the british all duping a into the...


In [5]:
import cv2
img_path = "/kaggle/input/memes-dataset/Dataset/"
def generate_img(img_path):
    path = str(img_path)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256))

    return img

imgs=[]
for index, row in df.iterrows():
    path = img_path + row['img']
    imgs.append(generate_img(path))
    
imgs = np.array(imgs)
imgs.shape

(8000, 256, 256, 3)

In [6]:
import re
import string 

nlp = spacy.load('en_core_web_sm')
slang_df = pd.read_csv('/kaggle/input/slangs-data/slangs_translation.csv')
slangs = dict(slang_df.values)
exclude = string.punctuation

def cleaning(data):
    
    df = data.copy()
    
    def remove_html_tags(text): return re.sub(r'<.*?>', '', text)
    df['text'] = df['text'].apply(remove_html_tags)
    
    def expand_contractions(text): return contractions.fix(text)
    df['text'] = df['text'].apply(expand_contractions)

    def remove_web_urls(text): return re.sub(r'https?:\/\/\S+', ' ', text)
    df['text'] = df['text'].apply(remove_web_urls)
    
    def remove_links(text): return re.sub(r'www\.[a-z]?\.?(com)+|[a-z]+\.(com)', ' ', text)
    df['text'] = df['text'].apply(remove_links)
    
    df['text'] = df['text'].str.lower()
    
    def chat_conversion(text):
        new_text = []
        for word in text.split():
            if word.lower() in slangs:
                new_text.append(slangs[word])
            else:
                new_text.append(word)
        return " ".join(new_text)
    
    df['text'] = df['text'].apply(chat_conversion)
    
    def remove_tags(text): return re.sub(r'@\w*', ' ' , text)
    df['text'] = df['text'].apply(remove_tags)
    
    def remove_hashtags(text): return re.sub(r'#\w*', ' ' , text)
    df['text'] = df['text'].apply(remove_hashtags)
    
    def remove_apostrophe(text): return re.sub(r"'s\b", "", text)
    df['text'] = df['text'].apply(remove_apostrophe)

    def remove_special_chars(text): return re.sub(r"[^a-zA-Z0-9\s]", ' ', text)
    df['text'] = df['text'].apply(remove_special_chars)
    
    def remove_number(text): return re.sub(r'[\d]', ' ', text)
    df['text'] = df['text'].apply(remove_number)
    
    def remove_punc(text): return ''.join([c for c in text if c not in exclude])
    df['text'] = df['text'].apply(remove_punc)
    
    def lemmatize(text):
        doc = nlp(text)
        lemmatized_tokens = [token.lemma_ for token in doc]
        return ' '.join(lemmatized_tokens)
    
    df['text'] = df['text'].apply(lemmatize)

    return df

# Apply function to dataframe
df = cleaning(df)
df.head(7)

,text,img,label,Caption
6252,I do nazi that bird fly by do you,img/63915.png,0,a black and white photo of a man with a mustache
4684,good holiday gift for muslim,img/37864.png,0,there is a man and woman sitting at a table wi...
1731,find a picture of your mother womb,img/90625.png,1,there is a baby that is laying down on a blanket
4742,bruce jenner cat,img/12807.png,0,arafed cat sitting in front of a building with...
4521,I do not want time out hear anymore about gun ...,img/26049.png,0,arafed image of a man in a suit and tie with a...
6340,normie your meme be racist and offensive ...,img/24537.png,1,there is a kid that is playing with a spongebob
576,information technology be tear in my eye time ...,img/35216.png,1,someone told the british all duping a into the...


In [7]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, BatchNormalization, LeakyReLU, Flatten, Dense, Reshape, Dropout, Add
from tensorflow.keras.models import Model

def residual_block(X,filters):
    # Retrieve Filters
    F1, F2 = filters
    # Saving the input value.we need this later to add to the output. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.1)(X)

    # Third component of main path 
    X = Conv2D(filters = F2, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization()(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = LeakyReLU(alpha=0.1)(X)
    return X

In [11]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Reshape


def create_bert_classification_model(dropout_rate=0.2, dense_layer_sizes=[128, 32], learning_rate=2e-6):
    base_model = ResNet50(
        weights='imagenet',
        input_shape=(256, 256, 3),  # Input shape of the images (height, width, channels)
        include_top=False  # Exclude the top classification layers
    )

    # Freeze the base model's layers to prevent them from being trained
    base_model.trainable = False
    
    img_input = tf.keras.layers.Input(shape=(256,256,3), dtype=tf.float32, name="img_input")
    
    x = base_model(img_input)

    # Add a convolutional layer with 512 filters, followed by batch normalization and LeakyReLU activation
    x = Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)

    # Add a custom residual block (function residual_block needs to be defined separately)
    x = residual_block(x, [64, 128])
    x = Flatten()(x)
    
    
    # Input layers for two text columns
    text_input1 = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text_input1")
    text_input2 = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text_input2")

    # Preprocessing for text column 1
    preprocessor1 = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
    encoder_inputs1 = preprocessor1(text_input1)
    
    # Preprocessing for text column 2
    preprocessor2 = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
    encoder_inputs2 = preprocessor2(text_input2)
    
    # BERT Encoder for each text column
    encoder1 = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",
        trainable=True)
    encoder2 = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",
        trainable=True)
    outputs1 = encoder1(encoder_inputs1)
    outputs2 = encoder2(encoder_inputs2)

    # Extract sequence outputs from BERT for each text column
    sequence_output1 = outputs1["sequence_output"]  # [batch_size, seq_length, 768].
    sequence_output2 = outputs2["sequence_output"]  # [batch_size, seq_length, 768].
    
    # Bidirectional LSTM layer for each text column
    lstm_output1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128))(sequence_output1)
    lstm_output2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128))(sequence_output2)


    concatenated_output = tf.keras.layers.Concatenate(axis=1)([x, lstm_output1, lstm_output2])
    hidden = tf.keras.layers.Dense(128, activation='relu')(concatenated_output)
    
    # Output layer
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(hidden)

    # Define and compile model
    model = tf.keras.Model(inputs=[img_input, text_input1, text_input2], outputs=[output_layer])
    
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=[tf.keras.metrics.BinaryAccuracy()])
    
    return model

# Example usage
model = create_bert_classification_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 img_input (InputLayer)      [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 resnet50 (Functional)       (None, 8, 8, 2048)           2358771   ['img_input[0][0]']           
                                                          2                                       
                                                                                                  
 conv2d_3 (Conv2D)           (None, 8, 8, 128)            2359424   ['resnet50[0][0]']            
                                                                                                  
 batch_normalization_3 (Bat  (None, 8, 8, 128)            512       ['conv2d_3[0][0]']      

In [12]:
history = model.fit(
    [imgs, df["text"], df["Caption"]],  
    df["label"],  # Target column
    epochs=5,
    batch_size=8
)

Epoch 1/5
1000/1000 [==============================] - 421s 345ms/step - loss: 0.7470 - binary_accuracy: 0.5656
Epoch 2/5
1000/1000 [==============================] - 340s 340ms/step - loss: 0.5915 - binary_accuracy: 0.6950
Epoch 3/5
1000/1000 [==============================] - 340s 340ms/step - loss: 0.4962 - binary_accuracy: 0.7675
Epoch 4/5
1000/1000 [==============================] - 340s 340ms/step - loss: 0.4193 - binary_accuracy: 0.8161
Epoch 5/5
1000/1000 [==============================] - 340s 340ms/step - loss: 0.3548 - binary_accuracy: 0.8539


In [15]:
history = model.fit(
    [imgs, df["text"], df["Caption"]],  
    df["label"],  # Target column
    epochs=2,
    batch_size=8
)

Epoch 1/2
1000/1000 [==============================] - 340s 340ms/step - loss: 0.2968 - binary_accuracy: 0.8844
Epoch 2/2
1000/1000 [==============================] - 340s 340ms/step - loss: 0.2428 - binary_accuracy: 0.9135


In [16]:
# EVALUATION

import cv2
img_path = "/kaggle/input/memes-dataset/Dataset/"
def generate_img(img_path):
    path = str(img_path)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256))

    return img


test_img =[]
for index, row in test.iterrows():
    path = img_path + row['img']
    test_img.append(generate_img(path))
    
test_img = np.array(test_img)

test = cleaning(test)

In [17]:
# Assuming you have your test data in text_column1_test, text_column2_test, and target_test
column1 = test['text']
column2 = test['Caption']
target = test['label']

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate([test_img, column1, column2], target, verbose=2)

# Predict on test data
y_pred = model.predict([test_img, column1, column2])

binary_threshold = 0.5
binary_y_pred = (y_pred >= binary_threshold).astype(int)

auc_score = roc_auc_score(target, binary_y_pred)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print("AUC Score", round(auc_score, 4))

63/63 - 29s - loss: 0.7090 - binary_accuracy: 0.6875 - 29s/epoch - 459ms/step
63/63 [==============================] - 29s 460ms/step
Test Loss: 0.7089815735816956
Test Accuracy: 0.6875
AUC Score 0.6628


In [18]:
# EVALUATION ON COVID19 DATASET

import pandas as pd
data = pd.read_csv('/kaggle/input/brain-dead-harmful-data-for-covid19/harmful_data_for_covid19/haemful_meme_for_covid19.csv', encoding= 'ISO-8859-1')
data.head(7)

,image,labels,text
0,covid_memes_1466.png,harmful,"""You are what you eat""\nSome dude in wuhan:\nI..."
1,covid_memes_471.png,harmful,COVID FACTS:\nPer the CDC elfective 07/10/2020...
2,covid_memes_1413.png,not harmful,"NO00, DO NOT\nWEH BE LIKE\n"
3,covid_memes_1712.png,harmful,WHEN YOUR BARBER\nCOUGHS\n
4,covid_memes_3734.png,harmful,The Volaile Mermaid\nTFW you hear that Trump a...
5,covid_memes_5557.png,harmful,Colin Snoka\nTrump after receiving his Covid d...
6,covid_memes_2166.png,not harmful,BLINDED BY\nTHE RIGHT\nMERICA\nREVVED UP\nLIKE...


In [22]:
import tensorflow as tf
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

# https://huggingface.co/Salesforce/blip-image-captioning-large
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

for index, row in data.iterrows():
    img = row.img
    root_path = "/kaggle/input/brain-dead-harmful-data-for-covid19/harmful_data_for_covid19/"
    raw_image = Image.open(root_path+str(img)).convert('RGB')
    
    # unconditional image captioning
    inputs = processor(raw_image, return_tensors="pt").to("cuda")
    out = model.generate(**inputs, max_new_tokens=40)
    caption = processor.decode(out[0], skip_special_tokens=True)
    data.at[index, 'Caption'] = caption
    raw_image = Image.open(root_path+str(img)).convert('RGB')

GPU is available


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 15.89 GiB of which 26.12 MiB is free. Process 2956 has 15.87 GiB memory in use. Of the allocated memory 341.62 MiB is allocated by PyTorch, and 4.38 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import cv2
img_path = "/kaggle/input/memes-dataset/Dataset/"
def generate_img(img_path):
    path = str(img_path)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256))

    return img

covid =[]
for index, row in df.iterrows():
    path = img_path + row['img']
    covid.append(generate_img(path))
    
covid = np.array(covid)
covid.shape

In [ ]:
# Assuming you have your test data in text_column1_test, text_column2_test, and target_test
column1 = data['text']
column2 = data['Caption']
target = data['label']

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate([covid, column1, column2], target, verbose=2)

# Predict on test data
y_pred = model.predict([covid, column1, column2])

binary_threshold = 0.5
binary_y_pred = (y_pred >= binary_threshold).astype(int)

auc_score = roc_auc_score(target, binary_y_pred)

printf("COVID 19 data:")
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print("AUC Score", round(auc_score, 4))

In [ ]:
# EVALUATION ON COVID19 DATASET

import pandas as pd
data = pd.read_csv('/kaggle/input/brain-dead-harmful-data-for-politics/harmful_meme_politics/harmful_meme_for_politics.csv', encoding= 'ISO-8859-1')
data.head(7)

In [ ]:
import tensorflow as tf
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

# https://huggingface.co/Salesforce/blip-image-captioning-large
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

for index, row in data.iterrows():
    img = row.img
    root_path = "/kaggle/input/brain-dead-harmful-data-for-politics/harmful_meme_politics/"
    raw_image = Image.open(root_path+str(img)).convert('RGB')
    
    # unconditional image captioning
    inputs = processor(raw_image, return_tensors="pt").to("cuda")
    out = model.generate(**inputs, max_new_tokens=40)
    caption = processor.decode(out[0], skip_special_tokens=True)
    data.at[index, 'Caption'] = caption"
    raw_image = Image.open(root_path+str(img)).convert('RGB')

In [ ]:
import cv2
img_path = "/kaggle/input/memes-dataset/Dataset/"
def generate_img(img_path):
    path = str(img_path)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256))

    return img

politics =[]
for index, row in df.iterrows():
    path = img_path + row['img']
    politics.append(generate_img(path))
    
politics = np.array(politics)
politics.shape

In [ ]:
# Assuming you have your test data in text_column1_test, text_column2_test, and target_test
column1 = data['text']
column2 = data['Caption']
target = data['label']

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate([covid, column1, column2], target, verbose=2)

# Predict on test data
y_pred = model.predict([covid, column1, column2])

binary_threshold = 0.5
binary_y_pred = (y_pred >= binary_threshold).astype(int)

auc_score = roc_auc_score(target, binary_y_pred)

printf("COVID 19 data:")
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print("AUC Score", round(auc_score, 4))